In [13]:
%matplotlib inline

import jsonlines
from pathlib import Path
from hydra import initialize, compose
import torchvision

import dr_util.file_utils as fu
import dr_gen.utils.model as mu
import dr_gen.utils.run as ru

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Setup Config and Generator

In [2]:
with initialize(config_path="../configs/", version_base=None):
    cfg = compose(
        config_name="config.yaml",
    )

In [3]:
generator = ru.set_deterministic(cfg.seed)

### Find Relevant Logs

In [14]:
def get_all_logs(base_dir):
    log_dir = Path(base_dir)
    all_files = [f for f in log_dir.rglob("*.jsonl") if f.is_file()]
    all_file_data = {str(f): fu.load_file(str(f)) for f in all_files}
    return all_file_data

In [18]:
def parse_run_log(log_data):
    run_cfg = log_data[0].get("value", {})
    metrics = {k: [] for k in ['train', 'val', 'eval']}
    for l in log_data[1:]:
        data_name = l.get("data_name", None)
        if data_name in metrics:
            metrics[data_name].append(l)
    return run_cfg, metrics

In [15]:
run_logs = get_all_logs("/scratch/ddr8143/logs")

In [19]:
test_rc, test_m = parse_run_log(next(iter(run_logs.items()))[1])

In [22]:
test_m.keys()

dict_keys(['train', 'val', 'eval'])